In [2]:
%pip install tweepy

Defaulting to user installation because normal site-packages is not writeable
  Using cached tweepy-4.14.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl.metadata (10 kB)
Using cached tweepy-4.14.0-py3-none-any.whl (98 kB)
Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tweepy
import pandas as pd
import json
from datetime import datetime, timedelta

# Load credentials from the .json file
with open("twitter_cred.json", "r") as file:
    creds = json.load(file)

# Function to authenticate with a given account's credentials
def authenticate(account_name):
    auth = tweepy.OAuth1UserHandler(
        creds[account_name]["API_KEY"],
        creds[account_name]["API_SECRET"],
        creds[account_name]["ACCESS_TOKEN"],
        creds[account_name]["ACCESS_TOKEN_SECRET"]
    )
    return tweepy.API(auth)

# Function to fetch popular tweets for a given query
def fetch_popular_tweets(api, query, max_tweets=100):  # Limit to 100 tweets per account
    tweets = []
    try:
        for tweet in tweepy.Cursor(
            api.search_tweets,
            q=query,
            lang="en",  # Restrict to English
            result_type="popular"  # Focus on tweets with high engagement
        ).items(max_tweets):
            tweets.append({
                "created_at": tweet.created_at,
                "text": tweet.text,
                "user": tweet.user.screen_name,
                "retweets": tweet.retweet_count,
                "likes": tweet.favorite_count,
                "id": tweet.id
            })
    except tweepy.errors.TweepyException as e:
        print(f"Error fetching tweets: {e}")
    return tweets

# Function to handle date assignments to accounts
def assign_dates_to_accounts(start_date, num_days_per_account):
    date_assignments = {}
    # Distribute 3 consecutive days across each of 5 accounts
    for i in range(5):  # 5 accounts
        account_name = f"account{i+1}"
        assigned_dates = []
        for j in range(num_days_per_account):  # Each account gets 3 days
            assigned_dates.append(start_date + timedelta(days=i * num_days_per_account + j))
        date_assignments[account_name] = assigned_dates
    return date_assignments

# Main Script
start_date = datetime(2023, 2, 10)  # Set to the game release date (Feb 10, 2023)
num_days_per_account = 3  # Each account gets 3 days of data

# Assign specific dates to each account
date_assignments = assign_dates_to_accounts(start_date, num_days_per_account)

# Loop through each account and fetch data for the assigned dates
tweets_all = []
for account_name, dates in date_assignments.items():
    # Authenticate with the current account
    api = authenticate(account_name)
    
    for date in dates:
        # Prepare the query for the given date
        query = f'"Hogwarts Legacy" OR #HogwartsLegacy lang:en since:{date.date()} until:{(date + timedelta(days=1)).date()}'
        print(f"Fetching tweets for {account_name} on {date.date()}")
        
        # Fetch tweets for that day (up to the 100 tweet limit per account for the month)
        tweets = fetch_popular_tweets(api, query, max_tweets=100)  # Limit to 100 tweets per account
        tweets_all.extend(tweets)

# Save all collected tweets to a CSV file
df = pd.DataFrame(tweets_all)
df.to_csv("hogwarts_legacy_popular_tweets.csv", index=False)
print(f"Saved {len(tweets_all)} popular tweets to 'hogwarts_legacy_popular_tweets.csv'.")
